In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Cs412/project_files

/content/drive/MyDrive/Cs412/project_files


In [ ]:
%ls

In [3]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 1.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, precision_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import traceback
import numpy as np
from scipy.sparse import isspmatrix, csr_matrix
import pickle

In [5]:
data = pd.read_csv('bugs-train.csv')
# Load test data
test_data = pd.read_csv('bugs-test.csv')
# Severity mapping
severity_mapping = {
    'trivial': 0,
    'enhancement': 1,
    'minor': 2,
    'normal': 3,
    'major': 4,
    'blocker': 5,
    'critical': 6
}
# Map severity values to integers
data['severity'] = data['severity'].map(severity_mapping)


In [12]:
# Extract features using BERT
X = data['summary'].tolist()
y = data['severity'].values
X_test = test_data['summary'].tolist()

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the text data in batches to manage memory usage
def batch_encode_texts(texts, batch_size=64):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = model_output.last_hidden_state[:, 0, :].numpy()
        embeddings.append(batch_embeddings)
        print( f"Completed batch {i // batch_size + 1}/{len(texts) // batch_size + 1}")
    return np.vstack(embeddings)

In [7]:
X_encoded = batch_encode_texts(X)

Completed batch 1/2500
Completed batch 2/2500
Completed batch 3/2500
Completed batch 4/2500
Completed batch 5/2500
Completed batch 6/2500
Completed batch 7/2500
Completed batch 8/2500
Completed batch 9/2500
Completed batch 10/2500
Completed batch 11/2500
Completed batch 12/2500
Completed batch 13/2500
Completed batch 14/2500
Completed batch 15/2500
Completed batch 16/2500
Completed batch 17/2500
Completed batch 18/2500
Completed batch 19/2500
Completed batch 20/2500
Completed batch 21/2500
Completed batch 22/2500
Completed batch 23/2500
Completed batch 24/2500
Completed batch 25/2500
Completed batch 26/2500
Completed batch 27/2500
Completed batch 28/2500
Completed batch 29/2500
Completed batch 30/2500
Completed batch 31/2500
Completed batch 32/2500
Completed batch 33/2500
Completed batch 34/2500
Completed batch 35/2500
Completed batch 36/2500
Completed batch 37/2500
Completed batch 38/2500
Completed batch 39/2500
Completed batch 40/2500
Completed batch 41/2500
Completed batch 42/2500
C

In [13]:
X_test_encoded = batch_encode_texts(X_test)

Completed batch 1/1346
Completed batch 2/1346
Completed batch 3/1346
Completed batch 4/1346
Completed batch 5/1346
Completed batch 6/1346
Completed batch 7/1346
Completed batch 8/1346
Completed batch 9/1346
Completed batch 10/1346
Completed batch 11/1346
Completed batch 12/1346
Completed batch 13/1346
Completed batch 14/1346
Completed batch 15/1346
Completed batch 16/1346
Completed batch 17/1346
Completed batch 18/1346
Completed batch 19/1346
Completed batch 20/1346
Completed batch 21/1346
Completed batch 22/1346
Completed batch 23/1346
Completed batch 24/1346
Completed batch 25/1346
Completed batch 26/1346
Completed batch 27/1346
Completed batch 28/1346
Completed batch 29/1346
Completed batch 30/1346
Completed batch 31/1346
Completed batch 32/1346
Completed batch 33/1346
Completed batch 34/1346
Completed batch 35/1346
Completed batch 36/1346
Completed batch 37/1346
Completed batch 38/1346
Completed batch 39/1346
Completed batch 40/1346
Completed batch 41/1346
Completed batch 42/1346
C

##Imported without test

In [9]:
# Save encoded texts and labels
with open('encoded_texts_512.pkl', 'wb') as f:
    pickle.dump((X_encoded, y), f)



In [ ]:
# Load encoded texts and labels (for future use)
with open('encoded_texts_512.pkl', 'rb') as f:
    X_encoded, y = pickle.load(f)

## Imported with test

In [14]:
# Save encoded texts and labels
with open('encoded_texts_train_test_512.pkl', 'wb') as f:
    pickle.dump((X_encoded, y, X_test_encoded), f)


In [ ]:

# Load encoded texts and labels (for future use)
with open('encoded_texts_train_test_512.pkl', 'rb') as f:
    X_encoded, y, X_test_encoded = pickle.load(f)

In [16]:
print(X_test_encoded.shape)

(86094, 768)


## GridSeacrh

In [ ]:
from sklearn.model_selection import ParameterGrid
# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define SMOTE
smote = SMOTE(sampling_strategy={
    0: 7000,  # trivial
    1: 15000,  # enhancement
    2: 10000,  # minor
    4: 20000,  # major
    5: 5000,   # blocker
    6: 50000  # critical
}, random_state=42)

# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
best_score = 0
best_params = None

# Scoring method
scorer = make_scorer(precision_score, average='macro')

try:
    for params in ParameterGrid(param_grid):
        print(f"Testing parameters: {params}")
        scores = []

        # Initialize the Random Forest classifier with current parameters
        clf = RandomForestClassifier(**params, verbose=2, n_jobs=-1, random_state=42)

        for train_index, test_index in skf.split(X_encoded, y):
            X_train, X_test = X_encoded[train_index], X_encoded[test_index]
            y_train, y_test = y[train_index], y[test_index]

            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            clf.fit(X_train_res, y_train_res)
            y_pred = clf.predict(X_test)

            score = precision_score(y_test, y_pred, average='macro')
            scores.append(score)
            print(f"Fold score: {score}")

            # Print classification report for this fold
            print(classification_report(y_test, y_pred))

        avg_score = np.mean(scores)
        print(f"Average Macro Precision for current parameters: {avg_score}")

        if avg_score > best_score:
            best_score = avg_score
            best_params = params

    print(f"Best Parameters: {best_params}")
    print(f"Best Macro Precision Score: {best_score}")

except Exception as e:
    print(f"Error evaluating Random Forest: {e}")
    traceback.print_exc()


Testing parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100building tree 12 of 100

building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  11 out of 100 | elapsed:   16.4s remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  62 out of 100 | elapsed:   17.3s remaining:   10.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.1s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  11 out of 100 | elapsed:    0.1s remaining:    0.9s
[Parallel(n_jobs=96)]: Done  62 out of 100 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.2s finished


Fold score: 0.6961103842634583
              precision    recall  f1-score   support

           0       0.40      0.00      0.01       401
           1       0.63      0.02      0.04      1475
           2       0.50      0.00      0.01      1034
           3       0.86      0.98      0.91     41952
           4       0.70      0.04      0.08      2018
           5       1.00      0.02      0.04       234
           6       0.78      0.67      0.72      6219

    accuracy                           0.85     53333
   macro avg       0.70      0.25      0.26     53333
weighted avg       0.83      0.85      0.81     53333



...

building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300building tree 38 of 300

building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
b

[Parallel(n_jobs=-1)]: Done 260 out of 300 | elapsed:   42.5s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   49.1s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done 260 out of 300 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=96)]: Done 300 out of 300 | elapsed:    0.5s finished


Fold score: 0.6761616651504875
              precision    recall  f1-score   support

           0       0.67      0.01      0.02       402
           1       0.56      0.02      0.03      1476
           2       0.33      0.00      0.01      1034
           3       0.86      0.97      0.91     41951
           4       0.74      0.04      0.07      2017
           5       0.80      0.02      0.03       233
           6       0.78      0.69      0.73      6220

    accuracy                           0.85     53333
   macro avg       0.68      0.25      0.26     53333
weighted avg       0.82      0.85      0.81     53333



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
b

RF

In [10]:
# Apply SMOTE to increase data points for specified classes

smote = SMOTE(sampling_strategy={
    0: 7000,  # trivial
    1: 15000,  # enhancement
    2: 10000,  # minor
    4: 20000,  # major
    5: 5000,   # blocker
    6 : 50000  # critical
}, random_state=42)



# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []



# Initialize the Random Forest classifier
clf = RandomForestClassifier(verbose=2, n_jobs=-1, n_estimators=200, criterion="entropy")

# Scoring method
scorer = make_scorer(precision_score, average='macro')



try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        clf.fit(X_train_res, y_train_res)
        y_pred = clf.predict(X_test)

        score = precision_score(y_test, y_pred, average='macro')
        scores.append(score)
        print(f"Fold score: {score}")

        # Print classification report for this fold
        print(classification_report(y_test, y_pred))

    print(f"Random Forest Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating Random Forest: {e}")
    traceback.print_exc()


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200building tree 11 of 200

building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200

building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=-1)]: Done 110 out of 200 | elapsed:   52.4s remaining:   42.9s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.1min finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done 110 out of 200 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 200 out of 200 | elapsed:    0.4s finished


Fold score: 0.683717445788384
              precision    recall  f1-score   support

           0       0.40      0.00      0.01       401
           1       0.62      0.02      0.03      1475
           2       0.42      0.00      0.01      1034
           3       0.86      0.97      0.91     41952
           4       0.71      0.04      0.08      2018
           5       1.00      0.01      0.03       234
           6       0.78      0.69      0.73      6219

    accuracy                           0.85     53333
   macro avg       0.68      0.25      0.26     53333
weighted avg       0.83      0.85      0.81     53333



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


...

Fold score: 0.7271441199071508
              precision    recall  f1-score   support

           0       0.75      0.01      0.03       402
           1       0.64      0.02      0.03      1476
           2       0.53      0.01      0.02      1034
           3       0.86      0.98      0.91     41951
           4       0.72      0.04      0.08      2017
           5       0.80      0.02      0.03       233
           6       0.78      0.69      0.73      6220

    accuracy                           0.85     53333
   macro avg       0.73      0.25      0.26     53333
weighted avg       0.83      0.85      0.81     53333



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200building tree 7 of 200
building tree 8 of 200

building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=-1)]: Done 110 out of 200 | elapsed:   52.8s remaining:   43.2s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.1min finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done 110 out of 200 | elapsed:    0.5s remaining:    0.4s
[Parallel(n_jobs=96)]: Done 200 out of 200 | elapsed:    0.6s finished


Fold score: 0.7271993168000123
              precision    recall  f1-score   support

           0       0.67      0.01      0.02       401
           1       0.63      0.02      0.03      1475
           2       0.78      0.01      0.01      1034
           3       0.86      0.98      0.91     41951
           4       0.74      0.04      0.08      2018
           5       0.64      0.03      0.06       234
           6       0.78      0.69      0.73      6219

    accuracy                           0.85     53332
   macro avg       0.73      0.25      0.26     53332
weighted avg       0.83      0.85      0.81     53332

Random Forest Macro Precision: 0.712686960831849


In [ ]:

# Train on the entire dataset
try:
    X_train_res, y_train_res = smote.fit_resample(X_encoded, y)
    clf.fit(X_train_res, y_train_res)
except Exception as e:
    print(f"Error training Random Forest on the entire dataset: {e}")
    traceback.print_exc()

# Make predictions on the test dataset
try:
    test_predictions = clf.predict(X_test_encoded)
except Exception as e:
    print(f"Error making predictions on the test dataset: {e}")
    traceback.print_exc()




In [ ]:


# Make predictions on the test dataset
test_predictions = clf.predict(X_test_encoded)



# Inverse mapping
inverse_severity_mapping = {v: k for k, v in severity_mapping.items()}
test_data['severity_mapped'] = test_predictions
test_data['severity'] = test_data['severity_mapped'].map(inverse_severity_mapping)

# Save the test predictions
test_data.rename(columns={"bug id": "bug_id"}, inplace=True)
output_data = test_data[['bug_id', 'severity']]
output_file_path = 'rf_bert_predictions_6.csv'
output_data.to_csv(output_file_path, index=False)

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done 110 out of 200 | elapsed:    0.6s remaining:    0.5s
[Parallel(n_jobs=96)]: Done 200 out of 200 | elapsed:    0.7s finished
